# Data Cleaning for Reddit Autos Selfposts<div class="tocSkip">
    
&copy; Jens Albrecht, 2021
    
This notebook can be freely copied and modified.  
Attribution, however, is highly appreciated.

<hr/>

See also: 

Albrecht, Ramachandran, Winkler: **Blueprints for Text Analytics in Python** (O'Reilly 2020)  
Chapter 4: [Preparing Data for Statistics and Machine Learning](https://learning.oreilly.com/library/view/blueprints-for-text/9781492074076/ch04.html#ch-preparation) + [Link to Github](https://github.com/blueprints-for-text-analytics-python/blueprints-text/blob/master/README.md)

## Setup<div class='tocSkip'/>

Set directory locations. If working on Google Colab: copy files and install required libraries.

In [ ]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    GIT_ROOT = 'https://github.com/jsalbr/tdwi-2021-text-mining/raw/main'
    os.system(f'wget {GIT_ROOT}/notebooks/setup.py')

%run -i setup.py

## Load Python Settings<div class="tocSkip"/>

Common imports, defaults for formatting in Matplotlib, Pandas etc.

In [ ]:
%run "$BASE_DIR/notebooks/settings.py"

%reload_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'png'

# to print output of all statements and not just the last
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# otherwise text between $ signs will be interpreted as formula and printed in italic
pd.set_option('display.html.use_mathjax', False)
pd.options.plotting.backend = "matplotlib"

# path to import blueprints packages
sys.path.append('./packages')

# Loading the Data Set

In [ ]:
df = pd.read_csv(f"{BASE_DIR}/data/reddit-autos-selfposts-raw.csv", sep=";", decimal=".", parse_dates=['created'])

In [ ]:
df['subreddit'].value_counts().head(20).plot(kind='barh', height=500).update_yaxes(autorange="reversed")

In [ ]:
df.dtypes

In [ ]:
df.describe().T

In [ ]:
df.describe(include='O').T

# Cleaning Text Data


In [ ]:
text = """
After viewing the [PINKIEPOOL Trailer](https://www.youtu.be/watch?v=ieHRoHUg)
it got me thinking about the best match ups.
<lb>Here's my take:<lb><lb>[](/sp)[](/ppseesyou) Deadpool<lb>[](/sp)[](/ajsly)
Captain America<lb>"""

In [ ]:
print(text)

## Identify Noise with Regular Expressions


In [ ]:
import re

RE_SUSPICIOUS = re.compile(r'[&#<>{}\[\]\\]')

def impurity(text, min_len=10):
    """returns the share of suspicious characters in a text"""
    if text == None or len(text) < min_len:
        return 0
    else:
        return len(RE_SUSPICIOUS.findall(text))/len(text)

print(impurity(text))

In [ ]:
pd.options.display.max_colwidth = 200
# add new column to data frame
df['impurity'] = df['text'].progress_apply(impurity, min_len=10)

# get the top 3 records
df[['text', 'impurity']].sort_values(by='impurity', ascending=False).head(3)

## Noise Removal with Regular Expressions


In [ ]:
import html

def clean(text):
    # convert html escapes like &amp; to characters.
    text = html.unescape(text) 
    text = html.unescape(text) 
    # tags like <tab>
    text = re.sub(r'<[^<>]*>', ' ', text)
    # markdown URLs like [Some text](https://....)
    text = re.sub(r'\[([^\[\]]*)\]\([^\(\)]*\)', r'\1', text)
    # text or code in brackets like [0]
    text = re.sub(r'\[[^\[\]]*\]', ' ', text)
    # standalone sequences of specials, matches &# but not #cool
    text = re.sub(r'(?:^|\s)[&#<>{}\[\]+|\\:-]{1,}(?:\s|$)', ' ', text)
    # standalone sequences of hyphens like --- or ==
    text = re.sub(r'(?:^|\s)[\-=\+]{2,}(?:\s|$)', ' ', text)
    # sequences of white spaces
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

In [ ]:
clean_text = clean(text)
print(clean_text)
print("Impurity:", impurity(clean_text))

In [ ]:
df['clean_text'] = df['text'].progress_map(clean)
df['impurity']   = df['clean_text'].apply(impurity, min_len=20)

df[['clean_text', 'impurity']].sort_values(by='impurity', ascending=False).head(3)

## Character Normalization with textacy


In [ ]:
text = "The café “Saint-Raphaël” is loca-\nted on Côte dʼAzur."

In [ ]:
import textacy
import textacy.preprocessing as tprep

def normalize(text):
    text = tprep.normalize.hyphenated_words(text)
    text = tprep.normalize.quotation_marks(text)
    text = tprep.normalize.unicode(text)
    text = tprep.remove.accents(text)
    return text

print(normalize(text))

In [ ]:
df['clean_text'] = df['clean_text'].progress_map(normalize)

## Pattern-based Data Masking with textacy


In [ ]:
from textacy.preprocessing.resources import RE_URL
from blueprints.exploration import count_words

count_words(df, column='clean_text', preprocess=RE_URL.findall).head(3)

In [ ]:
from textacy.preprocessing.replace import urls as replace_urls

text = "Check out https://spacy.io/usage/spacy-101"

# using default substitution _URL_
print(replace_urls(text))

In [ ]:
df['clean_text'] = df['clean_text'].progress_map(replace_urls)

# Checkpoint: Save the Cleaned Data

In [ ]:
df['text'] = df['title'] + ': ' + df['clean_text']

df[['subreddit', 'text']].sample(5)

In [ ]:
df.drop(columns=['clean_text', 'impurity'], inplace=True)

In [ ]:
df.to_csv("reddit-autos-selfposts-cleaned.csv", sep=";", decimal=".", index=False)